Laboratorium 4:  Docker i konteneryzacja modelu ML

In [17]:
%%writefile requirements.txt
flask
numpy
scikit-learn
gunicorn

Overwriting requirements.txt


In [18]:
%%writefile C:\Users\julia\JupyterNotebook\Dockerfile
#Użycie lekkiego obrazu Pythona
FROM python:3.9-slim

#Ustawienie katalogu roboczego
WORKDIR /app

#Kopiowanie plików aplikacji
COPY . /app

#Instalacja zależności
RUN pip install --no-cache-dir -r requirements.txt

#Wystawienie portu
EXPOSE 5000

#Uruchomienie serwera Gunicorn
CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:5000", "app:app"]

Writing C:\Users\julia\JupyterNotebook\Dockerfile


In [19]:
%%writefile C:\Users\julia\JupyterNotebook\app.py
from flask import Flask, request, jsonify
import numpy as np
from sklearn.linear_model import LinearRegression

app = Flask(__name__)

# Przykładowe dane do trenowania modelu ML
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])
model = LinearRegression().fit(X, y)

@app.route("/", methods=["GET"])
def home():
    return jsonify({"student": "Julia Adamowicz"})

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    if "input" not in data:
        return jsonify({"error": "Brak wymaganej wartosci"}), 400
    try:
        input_value = np.array([[data["input"]]])
        prediction = model.predict(input_value).tolist()
        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Overwriting C:\Users\julia\JupyterNotebook\app.py


In [20]:
!curl http://localhost:5000

{"student":"Julia Adamowicz"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    30  100    30    0     0    128      0 --:--:-- --:--:-- --:--:--   128


In [21]:
!curl -X POST http://localhost:5000/predict -H "Content-Type: application/json" -d "{\"input\": 5}"

{"prediction":[10.0]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    34  100    22  100    12    105     57 --:--:-- --:--:-- --:--:--   163


In [21]:
!pip install redis


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%%writefile C:\Users\julia\JupyterNotebook\app.py
from flask import Flask, request, jsonify
import numpy as np
import redis
from sklearn.linear_model import LinearRegression

app = Flask(__name__)

# Połączenie z Redis (host "redis" zamiast "localhost", bo Docker używa nazw kontenerów)
redis_client = redis.Redis(host="redis-1", port=6379, decode_responses=True)

# Przykładowe dane do trenowania modelu ML
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])
model = LinearRegression().fit(X, y)

@app.route("/", methods=["GET"])
def home():
    return jsonify({"student": "Julia Adamowicz"})

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    if "input" not in data:
        return jsonify({"error": "Brak wymaganej wartosci"}), 400
    try:
        input_value = np.array([[data["input"]]])
        prediction = model.predict(input_value).tolist()
            
        # Zapis wyniku do Redis
        redis_client.set("last_prediction", prediction[0])

        return jsonify({"prediction": prediction})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

@app.route("/last", methods=["GET"])
def last_prediction():
    """ Pobiera ostatnią predykcję z Redis """
    last_pred = redis_client.get("last_prediction")
    if last_pred is None:
        return jsonify({"error": "Brak zapisanej predykcji"}), 404
    return jsonify({"last_prediction": float(last_pred)})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Writing C:\Users\julia\JupyterNotebook\app.py


In [2]:
!curl http://localhost:5000/

{"student":"Julia Adamowicz"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    30  100    30    0     0   1296      0 --:--:-- --:--:-- --:--:--  1304


In [3]:
!curl -X POST http://localhost:5000/predict -H "Content-Type: application/json" -d "{\"input\": 5}"

{"prediction":[10.0]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    34  100    22  100    12   1103    602 --:--:-- --:--:-- --:--:--  1789


In [7]:
!curl http://localhost:5000/last

{"last_prediction":10.0}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    25  100    25    0     0   2006      0 --:--:-- --:--:-- --:--:--  2083


In [1]:
!curl -X POST https://flask-ml-app-628335485123.us-central1.run.app/predict -H "Content-Type: application/json" -d "{\"input\": 5}"

{"prediction":[10.0]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    34  100    22  100    12     67     37 --:--:-- --:--:-- --:--:--   105


In [2]:
%%writefile C:\Users\julia\JupyterNotebook\Notebook\NTPD4\ML-CI-CD\test_app.py
import numpy as np
from model import train_and_predict, get_accuracy

def test_predictions_not_none():
    """
    Test 1: Sprawdza, czy otrzymujemy jakąkolwiek predykcję.
    """
    preds, _ = train_and_predict()
    assert preds is not None, "Predictions should not be None."

def test_predictions_length():
    """
    Test 2 (na maksymalną ocenę 5): Sprawdza, czy długość listy predykcji jest większa od 0 i czy odpowiada liczbie próbek testowych.
    """
    preds, y_test = train_and_predict()
    assert len(preds) > 0, "Lista predykcji powinna zawierać co najmniej jeden element."
    assert len(preds) == len(y_test), f"Długość predykcji ({len(preds)}) nie zgadza się z liczbą próbek testowych ({len(y_test)})."

def test_predictions_value_range():
    """
    Test 3 (na maksymalną ocenę 5): Sprawdza, czy wartości w predykcjach mieszczą się w spodziewanym zakresie: Dla zbioru Iris mamy 3 klasy (0, 1, 2).
    """
    preds, _ = train_and_predict()
    assert all(pred in [0, 1, 2] for pred in preds), f"Predykcje powinny być w zakresie 0-2, ale znaleziono: {set(preds)}"

def test_model_accuracy():
    """
    Test 4 (na maksymalną ocenę 5): Sprawdza, czy model osiąga co najmniej 70% dokładności.
    """
    accuracy = get_accuracy()
    assert accuracy >= 0.7, f"Dokładność modelu powinna wynosić co najmniej 70%, ale wynosi {accuracy:.2f}"

Writing C:\Users\julia\JupyterNotebook\Notebook\NTPD4\ML-CI-CD\test_app.py


In [3]:
!pytest test_app.py

============================= test session starts =============================
platform win32 -- Python 3.10.0, pytest-8.3.5, pluggy-1.5.0
rootdir: C:\Users\julia\JupyterNotebook\Notebook\NTPD4\ML-CI-CD
collected 0 items / 1 error

=================================== ERRORS ====================================
________________________ ERROR collecting test_app.py _________________________
ImportError while importing test module 'C:\Users\julia\JupyterNotebook\Notebook\NTPD4\ML-CI-CD\test_app.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
..\..\..\..\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py:126: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
test_app.py:2: in <module>
    from model import train_and_predict, get_accuracy
E   ModuleNotFoundError: No module named 'model'
=========================== short test summary info ===========================
ERROR test_app.py
!!!!!!!!!!!!!!!!!!! Interr